In [20]:
import pandas as pd
import sqlite3

In [24]:
conn = sqlite3.connect('dialogue_analysis.db')

In [22]:
episodes_df = pd.read_csv('Sample Data/episodes.csv')
organizations_monthly_df = pd.read_csv('Sample Data/organizations_monthly.csv')
cost_to_serve_monthly_df = pd.read_csv('Sample Data/cost_to_serve_monthly.csv')

In [25]:
episodes_df.to_sql('episodes', conn, if_exists='replace', index=False)
organizations_monthly_df.to_sql('organizations_monthly', conn, if_exists='replace', index=False)
cost_to_serve_monthly_df.to_sql('cost_to_serve_monthly', conn, if_exists='replace', index=False)

12

## Utilization Analysis

Previewing Tables

In [26]:
# episodes table
episodes_preview = pd.read_sql_query("SELECT * FROM episodes LIMIT 5;", conn)
print("Episodes Preview:")
display(episodes_preview)

# organizations_monthly table
org_monthly_preview = pd.read_sql_query("SELECT * FROM organizations_monthly LIMIT 5;", conn)
print("\nOrganizations Monthly Preview:")
display(org_monthly_preview)

# cost_to_serve_monthly table
cost_serve_monthly_preview = pd.read_sql_query("SELECT * FROM cost_to_serve_monthly LIMIT 5;", conn)
print("\nCost to Serve Monthly Preview:")
display(cost_serve_monthly_preview)

Episodes Preview:


,EPISODE_ID,ORGANIZATION_ID,EPISODE_CREATED_AT,PROGRAM,OUTCOME
0,d0d178b23004534a5ff6723959785ba20ce1971d,5,2023-04-10T12:31:41.107Z,primary_care,walkin_clinic
1,ea85cffb681a06788b364d9ce49b61649f8688a5,5,2023-05-26T12:19:44.878Z,primary_care,walkin_clinic
2,7624c12a4ef8847e1951c0918d7193a9c3326dc1,5,2023-06-02T11:22:07.744Z,primary_care,md_np_appointment
3,9d994173c5775829c6b6badd898972522f038794,5,2023-11-24T14:00:17.658Z,primary_care,mhs_coaching
4,6be180209d276715e9256a78e8a73b13fe6a460d,2,2023-02-25T12:20:49.917Z,primary_care,md_np_appointment



Organizations Monthly Preview:


,DATE_MONTH,ORGANIZATION_ID,ELIGIBLE_MEMBERS,PEPM,HAS_PRIMARY_CARE_SINCE,HAS_MENTAL_HEALTH_SINCE,HAS_EAP_SINCE
0,2023-10-01T00:00:00Z,6,185,7.47,2000-01-01T00:00:00Z,None,None
1,2023-06-01T00:00:00Z,6,219,7.47,2000-01-01T00:00:00Z,None,None
2,2023-01-01T00:00:00Z,1,268,15.71,2000-01-01T00:00:00Z,2021-01-19T22:58:24.2361Z,2021-11-01T16:05:28.0127Z
3,2023-03-01T00:00:00Z,1,272,15.71,2000-01-01T00:00:00Z,2021-01-19T22:58:24.2361Z,2021-11-01T16:05:28.0127Z
4,2023-09-01T00:00:00Z,4,1436,14.21,2000-01-01T00:00:00Z,2021-06-01T12:15:33.8532Z,2023-04-05T21:32:35.77486Z



Cost to Serve Monthly Preview:


,DATE_MONTH,COST_TO_SERVE_PRIMARY_CARE,COST_TO_SERVE_MENTAL_HEALTH,COST_TO_SERVE_EAP
0,2023-01-01,60.04,334.29,112.27
1,2023-02-01,62.90,312.29,111.19
2,2023-03-01,62.96,298.27,119.29
3,2023-04-01,63.77,320.80,120.93
4,2023-05-01,65.27,276.54,114.95


Number of episodes by category for the year 2023

In [27]:
query = """
SELECT PROGRAM, COUNT(*) AS EPISODES_COUNT
FROM episodes
WHERE ORGANIZATION_ID = 4
GROUP BY PROGRAM;
"""
df = pd.read_sql_query(query, conn)
display(df)

,PROGRAM,EPISODES_COUNT
0,eap,67
1,mental_health,112
2,primary_care,550


Episodes opened by month

In [28]:
query_monthly_episodes = """
SELECT
  strftime('%Y-%m', EPISODE_CREATED_AT) AS month,
  COUNT(EPISODE_ID) AS episodes_opened
FROM episodes
WHERE organization_id = 4
GROUP BY month
ORDER BY month;


"""

df = pd.read_sql_query(query_monthly_episodes, conn)
display(df)

,month,episodes_opened
0,2023-01,1
1,2023-02,1
2,2023-05,158
3,2023-06,89
4,2023-07,78
5,2023-08,76
6,2023-09,58
7,2023-10,90
8,2023-11,90
9,2023-12,88


Eligible members per month

In [29]:
query_monthly_members = """
SELECT
  strftime('%Y-%m', date_month) AS month,
  eligible_members
FROM organizations_monthly
WHERE organization_id = 4
GROUP BY month
ORDER BY month;
"""

df = pd.read_sql_query(query_monthly_members, conn)
display(df)

,month,ELIGIBLE_MEMBERS
0,2023-01,59
1,2023-02,43
2,2023-03,42
3,2023-04,43
4,2023-05,1478
5,2023-06,1466
6,2023-07,1453
7,2023-08,1429
8,2023-09,1436
9,2023-10,1497


Combining queries in order to obtain member's utilization rate

In [30]:
query_utilization_rate = """
WITH monthly_episodes AS (
  SELECT
    strftime('%Y-%m', EPISODE_CREATED_AT) AS month,
    COUNT(EPISODE_ID) AS episodes_opened
  FROM episodes
  WHERE organization_id = 4
  GROUP BY month
),
monthly_members AS (
  SELECT
    strftime('%Y-%m', date_month) AS month,
    eligible_members
  FROM organizations_monthly
  WHERE organization_id = 4
  GROUP BY month
)
SELECT
  me.month,
  me.episodes_opened,
  mm.eligible_members,
  ROUND(((me.episodes_opened * 1.0/ mm.eligible_members) * 100), 2) AS utilization_rate_percentage
FROM monthly_episodes me
JOIN monthly_members mm ON me.month = mm.month
ORDER BY me.month;
"""

df = pd.read_sql_query(query_utilization_rate, conn)
display(df)

,month,episodes_opened,eligible_members,utilization_rate_percentage
0,2023-01,1,59,1.69
1,2023-02,1,43,2.33
2,2023-05,158,1478,10.69
3,2023-06,89,1466,6.07
4,2023-07,78,1453,5.37
5,2023-08,76,1429,5.32
6,2023-09,58,1436,4.04
7,2023-10,90,1497,6.01
8,2023-11,90,1500,6.00
9,2023-12,88,1528,5.76


Trends in utilization

In [31]:
query_utilization_trends = """
WITH monthly_episodes AS (
  SELECT
    strftime('%Y-%m', EPISODE_CREATED_AT) AS month,
    COUNT(EPISODE_ID) AS episodes_opened
  FROM episodes
  WHERE ORGANIZATION_ID = 4
  GROUP BY month
),
monthly_members AS (
  SELECT
    strftime('%Y-%m', date_month) AS month,
    eligible_members
  FROM organizations_monthly
  WHERE ORGANIZATION_ID = 4
  GROUP BY month
),
utilization_rates AS (
  SELECT
    me.month,
    me.episodes_opened,
    mm.eligible_members,
    ROUND(((me.episodes_opened * 1.0 / mm.eligible_members) * 100), 2) AS utilization_rate_percentage
  FROM monthly_episodes me
  JOIN monthly_members mm ON me.month = mm.month
),
utilization_trends AS (
  SELECT
    month,
    utilization_rate_percentage,
    utilization_rate_percentage - LAG(utilization_rate_percentage) OVER (ORDER BY month) AS mom_change
  FROM utilization_rates
)
SELECT
  month,
  utilization_rate_percentage,
  mom_change,
  CASE
    WHEN mom_change > 0 THEN 'Increase'
    WHEN mom_change < 0 THEN 'Decrease'
    ELSE 'Stable'
  END AS trend
FROM utilization_trends
ORDER BY month;
"""

df = pd.read_sql_query(query_utilization_trends, conn)
display(df)

,month,utilization_rate_percentage,mom_change,trend
0,2023-01,1.69,NaN,Stable
1,2023-02,2.33,0.64,Increase
2,2023-05,10.69,8.36,Increase
3,2023-06,6.07,-4.62,Decrease
4,2023-07,5.37,-0.70,Decrease
5,2023-08,5.32,-0.05,Decrease
6,2023-09,4.04,-1.28,Decrease
7,2023-10,6.01,1.97,Increase
8,2023-11,6.00,-0.01,Decrease
9,2023-12,5.76,-0.24,Decrease


Potential causes for changes in utilization: Episodes by program type per month

In [32]:
query_changes_utilization_rate = """
SELECT
  strftime('%Y-%m', EPISODE_CREATED_AT) AS month,
  PROGRAM,
  COUNT(EPISODE_ID) AS episodes_count
FROM episodes
WHERE organization_id = 4
GROUP BY month, PROGRAM
ORDER BY month, PROGRAM;
"""

df = pd.read_sql_query(query_changes_utilization_rate, conn)
display(df)

,month,PROGRAM,episodes_count
0,2023-01,primary_care,1
1,2023-02,primary_care,1
2,2023-05,eap,21
3,2023-05,mental_health,27
4,2023-05,primary_care,110
5,2023-06,eap,7
6,2023-06,mental_health,19
7,2023-06,primary_care,63
8,2023-07,eap,9
9,2023-07,mental_health,9


Adjusting the query to create separate columns for each program type

In [35]:
query_changes_utilization_rate_pivot = """
SELECT
  strftime('%Y-%m', EPISODE_CREATED_AT) AS month,
  SUM(CASE WHEN PROGRAM = 'primary_care' THEN 1 ELSE 0 END) AS primary_care,
  SUM(CASE WHEN PROGRAM = 'eap' THEN 1 ELSE 0 END) AS eap,
  SUM(CASE WHEN PROGRAM = 'mental_health' THEN 1 ELSE 0 END) AS mental_health
FROM episodes
WHERE ORGANIZATION_ID = 4
GROUP BY month
ORDER BY month;
"""

df = pd.read_sql_query(query_changes_utilization_rate_pivot, conn)
display(df)

,month,primary_care,eap,mental_health
0,2023-01,1,0,0
1,2023-02,1,0,0
2,2023-05,110,21,27
3,2023-06,63,7,19
4,2023-07,60,9,9
5,2023-08,61,10,5
6,2023-09,44,2,12
7,2023-10,67,8,15
8,2023-11,67,5,18
9,2023-12,76,5,7


## Financial Analysis

Monthly costs to serve per program type

In [46]:
query_monthly_costs = """
WITH episodes_per_month AS (
  SELECT
    strftime('%Y-%m', EPISODE_CREATED_AT) AS month,
    SUM(CASE WHEN PROGRAM = 'primary_care' THEN 1 ELSE 0 END) AS primary_care_episodes,
    SUM(CASE WHEN PROGRAM = 'eap' THEN 1 ELSE 0 END) AS eap_episodes,
    SUM(CASE WHEN PROGRAM = 'mental_health' THEN 1 ELSE 0 END) AS mental_health_episodes
  FROM episodes
  WHERE ORGANIZATION_ID = 4
  GROUP BY month
)
SELECT
   e.month,
  (e.primary_care_episodes * c.cost_to_serve_primary_care) AS total_cost_primary_care,
  (e.eap_episodes * c.cost_to_serve_eap) AS total_cost_eap,
  (e.mental_health_episodes * c.cost_to_serve_mental_health) AS total_cost_mental_health,
  (e.primary_care_episodes * c.cost_to_serve_primary_care) +
  (e.eap_episodes * c.cost_to_serve_eap) +
  (e.mental_health_episodes * c.cost_to_serve_mental_health) AS total_cost_to_serve
FROM episodes_per_month e
INNER JOIN cost_to_serve_monthly c ON e.month = strftime('%Y-%m', c.date_month)
ORDER BY e.month;
"""

df = pd.read_sql_query(query_monthly_costs, conn)
display(df)

,month,total_cost_primary_care,total_cost_eap,total_cost_mental_health,total_cost_to_serve
0,2023-01,60.04,0.00,0.00,60.04
1,2023-02,62.90,0.00,0.00,62.90
2,2023-05,7179.70,2413.95,7466.58,17060.23
3,2023-06,4261.32,931.14,5548.19,10740.65
4,2023-07,3961.20,1144.71,2809.44,7915.35
5,2023-08,3993.06,1321.00,1505.25,6819.31
6,2023-09,2739.00,197.80,2807.52,5744.32
7,2023-10,4014.64,679.68,3097.50,7791.82
8,2023-11,3576.46,377.85,4988.16,8942.47
9,2023-12,4138.96,736.25,2706.27,7581.48
